In [218]:
import pandas as pd
import keras
import numpy as np
import os
import sys
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from spacy.vectors import Vectors
import spacy
nlp = spacy.load('en')
from tensorflow.contrib.learn.python import preprocessing
#nlp_vec = spacy.load('en_vectors_web_lg')
from keras.preprocessing.sequence import pad_sequences

In [217]:
from keras.models import Sequential
from keras.layers import Embedding,MaxPool1D,Dropout,Dense,Conv1D,Flatten,LSTM
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import one_hot

In [313]:
import h5py

In [12]:
model_path = 'data/models/'
%mkdir -p $model_path

In [13]:
train = pd.read_csv("TRAIN_info_SMS.csv")
test = pd.read_csv("DEV_info_SMS.csv")

In [14]:
trn = train.Message.values

In [133]:
trn.shape

(12000,)

In [96]:
labels = train.Label.values

In [275]:
len(labels)
labels_one_hot = one_hot(labels,12,lower=False)

AttributeError: 'numpy.ndarray' object has no attribute 'translate'

In [16]:
trn_lbls = train.Label.unique()

In [17]:
trn_lbls

array(['Delivery', 'Hotel', 'Payment', 'Appointment', 'Train', 'PickUp',
       'Flight', 'Bus', 'Movie', 'Reservation', 'Expiry', 'Cab'], dtype=object)

In [278]:
enc = LabelEncoder()
oh = OneHotEncoder(sparse=False)
labels_e = enc.fit_transform(labels)


In [279]:
labels_e = labels_e.reshape(len(labels_e),1)
labels_o = oh.fit_transform(labels_e)
labels_e,print(labels_o[0])

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]


(array([[3],
        [6],
        [8],
        ..., 
        [4],
        [2],
        [9]]), None)

In [18]:
enc = LabelEncoder()
trn_lbls_e = enc.fit_transform(trn_lbls)
trn_lbls_e

array([ 3,  6,  8,  0, 11,  9,  5,  1,  7, 10,  4,  2])

In [28]:
words = [[doc.text for doc in nlp(i) if not doc.is_stop and doc.pos_!="NUM"] for i in trn] 

In [29]:
len(words)

12000

In [33]:
words_fltr = [[i for i in token if i not in ['.',':','-',',','/','(',')','[',']']]
              for token in words]

In [41]:
max([len(i) for i in trn])

354

In [163]:
word_ids_joined = [' '.join(s) for s in words_fltr]
word_ids_joined

['Dear Guest Thanks choosing The Edison Order ID Delivery Enjoy % discount purchase Code NL99J9 T&C.',
 'Welcome Southern University Shreveport R.No Ch In Ch.out',
 'Bill dated Rs generated Aircel fixedline broadband relationship The reach Due date',
 'Bill dated Rs generated Reliance fixedline broadband relationship The reach Due date',
 'DEAR Zachary THANK YOU FOR YOUR BOOKING ORDER ID YOUR ORDER HAS BEEN ASSIGNED TO Andres(9756754488 AND APPOINTMENT FOR SAMPLE COLLECTION IS SCHEDULED ON FOR ASSISTANCE WHATSAPP US ON',
 'PNR:5672673362,TRAIN:2564,DOJ:2010 04,AC3,AMLI BHTR Dep:13:32 Ashton B7 Fare:2427,SC:10.0+PG+INS',
 'Appointment Dr Hector ANWATTI confirmed For queries Your appt delayed case doctor attending emergency',
 'Repair ref.no car JC22723666 opened ETD car est.repair amt Rs.534 Rgds Maha Veer Auto Agency',
 'The PNR INDIGO Flt 6X837 DMU PAB hrs   H6UJHN Tip Check PNR status book MakeMyTrip App!. Thank',
 'Welcome Telecaf R.No Ch In Ch.out',
 'PNR:5531657287,TRAIN:32241,DOJ

In [175]:
vocab_len = 100
p = preprocessing.text.VocabularyProcessor(vocab_len)
wordids = p.fit_transform(word_ids_joined)

In [176]:
word_ids = np.array(list(wordids))


In [177]:
word_ids[2]

array([27, 28, 29, 30, 31, 32, 33, 34,  5, 35, 36, 37,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [280]:
x_train,x_valid,y_train,y_valid = train_test_split(word_ids,labels_o,test_size=0.4)

In [281]:
x_train.shape,y_train.shape

((7200, 100), (7200, 12))

In [ ]:
[len()]

In [287]:
vocab_size = 17000
x_train = [[i if i<vocab_size else vocab_size-1 for i in l] for l in x_train]
x_valid = [[i if i<vocab_size else vocab_size-1 for i in l] for l in x_valid]

In [303]:
#CNN model

model = Sequential([
    Embedding(vocab_size,32,input_length=vocab_len),
    Dropout(0.25),
    Conv1D(activation='relu',filters=128,kernel_size=3,padding='same'),
    Dropout(0.25),
    MaxPool1D(),
    Flatten(),
    Dropout(0.75),
    Dense(100,activation='relu'),
    Dense(12,activation='softmax')
])


In [304]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.optimizer.lr=0.01

In [305]:
model.fit(x_train,y_train,batch_size=128,epochs=10,validation_data=(x_valid,y_valid))

Train on 7200 samples, validate on 4800 samples
Epoch 1/10
7200/7200 [==============================] - 7s 1ms/step - loss: 0.2667 - acc: 0.9321 - val_loss: 4.4754e-06 - val_acc: 1.0000
Epoch 2/10
7200/7200 [==============================] - 7s 967us/step - loss: 0.0059 - acc: 0.9982 - val_loss: 3.1061e-04 - val_acc: 0.9998
Epoch 3/10
7200/7200 [==============================] - 7s 956us/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0014 - val_acc: 0.9998
Epoch 4/10
7200/7200 [==============================] - 7s 1ms/step - loss: 0.0094 - acc: 0.9982 - val_loss: 0.0015 - val_acc: 0.9998
Epoch 5/10
7200/7200 [==============================] - 8s 1ms/step - loss: 0.0096 - acc: 0.9979 - val_loss: 2.4751e-07 - val_acc: 1.0000
Epoch 6/10
7200/7200 [==============================] - 7s 982us/step - loss: 0.0327 - acc: 0.9954 - val_loss: 4.2939e-04 - val_acc: 0.9998
Epoch 7/10
7200/7200 [==============================] - 7s 972us/step - loss: 0.0312 - acc: 0.9962 - val_loss: 1.1921e-07 - va

In [322]:

model.save_weights('data/models/weights1.h5')

ImportError: No module named '(h5py)'

In [211]:
model_ls = Sequential([
    Embedding(vocab_size, 32, input_length=vocab_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model_ls.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ls.summary()

/home/surya/DL/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/surya/DL/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(17000, 32, mask_zero=True, embeddings_regularizer=<keras.reg..., input_length=100)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/surya/DL/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, implementation=2)`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 32)           544000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 101       
Total params: 597,301
Trainable params: 597,301
Non-trainable params: 0
_________________________________________________________________


In [212]:
model_ls.fit(x_train,y_train,batch_size=128,epochs=10,validation_data=(x_valid,y_valid))

ValueError: You are passing a target array of shape (7200, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [207]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(trn)
xtrain_tfv =  tfv.transform(trn) 
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [210]:
clf.score(xtrain_tfv,labels)

1.0

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])